<a href="https://colab.research.google.com/github/Renatoeletronico/Exercicios-de-linguagem-natural/blob/main/An%C3%A1lise_de_sentimentos_e_classifica%C3%A7%C3%A3o_de_textos_em_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplos de roteiro

In [1]:
# importando as bibliotecas
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#certifique de baixar os recursos necessários da nltk
nltk.download('vader_lexicon')
nltk.download('punkt')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# inicializando o analizador de sentimentos
sid = SentimentIntensityAnalyzer()

# Definindo os textos para treinar o modelo
textos = [
    "Este filme é incrível! Eu adorei cada minuto.",
    "Odiei este filme. Foi uma perda de tempo.",
    "O filme foi ok, nada de especial.",
    "Estou muito feliz com o produto, funciona perfeitamente!",
    "Que serviço horrível! Nunca mais compro nessa loja."
]

In [3]:
# Analisando os sentimentos dos textos

for texto in textos:
    print(f"Texto: {texto}")
    ss = sid.polarity_scores(texto)
    print(f"Pontuação: {ss}")
    if ss['compound'] >= 0.05:
        print("Sentimento: Positivo")
    elif ss['compound'] <= -0.05:
        print("Sentimento: Negativo")
    else:
        print("Sentimento: Neutro")
    print("\n")

Texto: Este filme é incrível! Eu adorei cada minuto.
Pontuação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: Odiei este filme. Foi uma perda de tempo.
Pontuação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: O filme foi ok, nada de especial.
Pontuação: {'neg': 0.0, 'neu': 0.694, 'pos': 0.306, 'compound': 0.296}
Sentimento: Positivo


Texto: Estou muito feliz com o produto, funciona perfeitamente!
Pontuação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro


Texto: Que serviço horrível! Nunca mais compro nessa loja.
Pontuação: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentimento: Neutro




In [4]:
import numpy as np

def contar_palavras(avaliacoes, sentimento):
    """conta a frequência de cada palavra em avaliações de um dado sentimento."""
    contagem = {}
    for avaliacao, sent in avaliacoes:
        if sent == sentimento:
            for palavra in avaliacao.lower().split():
                if palavra in contagem:
                    contagem[palavra] += 1
                else:
                    contagem[palavra] = 1
    return contagem

def calcular_probabilidade_sentimento(avaliacoes, sentimento):
    """calcula a probabilidade de uma avaliação ser de um dado sentimento."""
    total_avaliacoes = len(avaliacoes)
    contagem_sentimento = sum(1 for _, sent in avaliacoes if sent == sentimento)
    return contagem_sentimento / total_avaliacoes

def calcular_probabilidade_palavra_dado_sentimento(palavra, contagem_palavras, total_palavras_sentimento, total_palavras_vocabulario, alpha=1):
    """calcula a probabilidade de uma palavra aparecer dado um sentimento (com suavização Laplace)."""
    contagem_palavra = contagem_palavras.get(palavra, 0)
    return (contagem_palavra + alpha) / (total_palavras_sentimento + alpha * total_palavras_vocabulario)


In [5]:
def classificar_avaliacao(avaliacao, avaliacoes):
    """Classifica o sentimento de uma nova avaliação usando Naive Bayes."""
    sentimentos = ["Positivo", "Negativo"]
    probabilidades_sentimento = {}

    # Calcular probabilidades a priori dos sentimentos
    probabilidades_sentimento["Positivo"] = calcular_probabilidade_sentimento(avaliacoes, "Positivo")
    probabilidades_sentimento["Negativo"] = calcular_probabilidade_sentimento(avaliacoes, "Negativo")

    # Contar palavras em cada sentimento
    contagem_palavras_positivo = contar_palavras(avaliacoes, "Positivo")
    contagem_palavras_negativo = contar_palavras(avaliacoes, "Negativo")

    # Calcular total de palavras em cada sentimento
    total_palavras_positivo = sum(contagem_palavras_positivo.values())
    total_palavras_negativo = sum(contagem_palavras_negativo.values())

    # Calcular tamanho do vocabulário
    vocabulario = set()
    for avaliacao_, _ in avaliacoes:
        for palavra in avaliacao_.lower().split():
            vocabulario.add(palavra)
    total_palavras_vocabulario = len(vocabulario)

    # Calcular probabilidade para cada sentimento
    for sentimento in sentimentos:
        probabilidade_total = probabilidades_sentimento[sentimento]
        contagem_palavras_sentimento = contagem_palavras_positivo if sentimento == "Positivo" else contagem_palavras_negativo
        total_palavras_sentimento = total_palavras_positivo if sentimento == "Positivo" else total_palavras_negativo

        for palavra in avaliacao.lower().split():
            probabilidade_palavra = calcular_probabilidade_palavra_dado_sentimento(
                palavra, contagem_palavras_sentimento, total_palavras_sentimento, total_palavras_vocabulario
            )
            probabilidade_total *= probabilidade_palavra

        probabilidades_sentimento[sentimento] = probabilidade_total

    # Classificar com base na maior probabilidade
    melhor_sentimento = max(probabilidades_sentimento, key=probabilidades_sentimento.get)
    return melhor_sentimento


In [6]:
avaliacoes = [
    ("Filme incrível! Adorei a atuação.", "Positivo"),
    ("Roteiro fraco, não recomendo.", "Negativo"),
    ("Atuação boa, mas filme longo.", "Positivo"),
    ("Péssimo! Perdi meu tempo.", "Negativo"),
]


In [10]:
# Nova avaliação a ser classificada
nova_avaliacao = input("O que você achou do filme 101 Dálmatas? ")

# Classificar a nova avaliação
sentimento = classificar_avaliacao(nova_avaliacao, avaliacoes)
print(f"A avaliação '{nova_avaliacao}' foi classificada como: {sentimento}")


O que você achou do filme 101 Dálmatas? bom
A avaliação 'bom' foi classificada como: Positivo


In [9]:
avaliacoes = [
    ("Filme incrível! Adorei a atuação, roteiro e direção.", "Positivo"),
    ("Roteiro fraco, atuação ruim, não recomendo. Perda de tempo.", "Negativo"),
    ("Atuação boa, mas filme um pouco longo e arrastado.", "Positivo"),
    ("Péssimo! Perdi meu tempo, dinheiro e paciência. Horrível.", "Negativo"),
    ("Amei! Elenco maravilhoso, fotografia impecável. Recomendo muito!", "Positivo"),
    ("Chato, previsível e sem graça. Saí no meio do filme.", "Negativo"),
    ("Uma obra-prima! Roteiro inteligente, atuações memoráveis.", "Positivo"),
    ("Cansativo, atuações forçadas, final decepcionante.", "Negativo"),
    ("Divertido e emocionante! Ótimo para assistir com a família.", "Positivo"),
    ("Clichê, sem originalidade, diálogos fracos. Não vale a pena.", "Negativo"),
    ("Surpreendente e envolvente! Me prendeu do início ao fim.", "Positivo"),
    ("Confuso, mal dirigido, edição amadora. Um desastre.", "Negativo"),
    ("Atuação excelente, história cativante, trilha sonora perfeita.", "Positivo"),
    ("Atuações medianas, história arrastada, trilha sonora irritante.", "Negativo"),
    ("Roteiro bem construído, diálogos afiados, direção impecável.", "Positivo"),
    ("Roteiro fraco, diálogos bobos, direção perdida.", "Negativo"),
    ("Uma experiência cinematográfica única! Recomendo a todos os amantes do cinema.", "Positivo"),
    ("Um filme esquecível, sem emoção e sem impacto.", "Negativo"),
    ("Comovente e inspirador! Me fez refletir sobre a vida.", "Positivo"),
    ("Decepcionante e frustrante! Não recomendo nem para passar o tempo.", "Negativo"),
    ("Ótimo, gostei muito",'Positivo')
]


# Desafios do roteiro